# Minnesota BMSB Analysis: `Modeling & Simulation`
##### Contributors: *Luke Zaruba*, *Mattie Gisselbeck*
##### Last Updated: 2023-04-22

In this notebook, BMSB spread is modeled through the use of Monte Carlo Simulation and a Huff (gravity) model.

In [1]:
# Import Packages
import os
os.environ['USE_PYGEOS'] = '0'

import sys

import arcgis
import arcpy
import geopandas as gpd
import pandas as pd
from scipy.stats import zscore
from sqlalchemy import create_engine
from geoalchemy2 import Geometry, WKTElement

# Append Path & Import Model/Database Modules
sys.path.append("..")

from bmsb.model import Simulation

### Generate Distance Lags & Export to CSVs

In [6]:
# Path to FC with Aggregated Data
full_gdb_path = r"C:\gitFiles\minnesota-bmsb-analysis\data\gdb\bmsb_analysis.gdb"

# Calculate Lags
arcpy.analysis.GenerateOriginDestinationLinks(
    os.path.join(full_gdb_path, "cities_attributed"),
    os.path.join(full_gdb_path, "cities_attributed"),
    os.path.join(full_gdb_path, "distances"),
    num_nearest=100,
    distance_unit="MILES"
)

In [7]:
# Path to Output Folder
output_dir = r"C:\gitFiles\minnesota-bmsb-analysis\data\model"

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Export Lags to CSVs
arcpy.conversion.ExportTable(
    os.path.join(full_gdb_path, "distances"),
    os.path.join(output_dir, "distances.csv"),
    field_mapping="Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,distances,Shape_Length,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,distances,ORIG_FID,-1,-1;DEST_FID "DEST_FID" true true false 4 Long 0 0,First,#,distances,DEST_FID,-1,-1;LINK_DIST "LINK_DIST" true true false 8 Double 0 0,First,#,distances,LINK_DIST,-1,-1",
)

# Export Cities to CSV
arcpy.conversion.ExportTable(
    os.path.join(full_gdb_path, "cities_attributed"),
    os.path.join(output_dir, "cities.csv"),
)

### Load Cities & Calculate Weights

In [2]:
# Load Cities from CSV to DF
cities_df = pd.read_csv("../data/model/cities.csv")

# Change Column Names
cities_df.columns = ["OBJECTID", "GNIS", "City",
       "County", "Population", "Shape Length",
       "Elevation: Range", "Elevation: Mean", "Elevation: StDev",
       "Elevation: Median", "Weather: Join Count",
       "Weather: Mean Max Temp", "Weather: Mean Min Temp", "Weather: Mean Precip",
       "Observations: Count", "LC: Urban",
       "LC: Ag", "LC: Natural", "SHAPELENGTH", "SHAPEAREA"
]

# Convert LC to Percentages
cities_df["LC: Urban %"] = cities_df["LC: Urban"] / (cities_df["LC: Urban"] + cities_df["LC: Ag"] + cities_df["LC: Natural"])
cities_df["LC: Ag %"] = cities_df["LC: Ag"] / (cities_df["LC: Urban"] + cities_df["LC: Ag"] + cities_df["LC: Natural"])
cities_df["LC: Natural %"] = cities_df["LC: Natural"] / (cities_df["LC: Urban"] + cities_df["LC: Ag"] + cities_df["LC: Natural"])

# Convert Observation Count to Presence
cities_df["Observations: Presence"] = cities_df["Observations: Count"].apply(lambda x: 1 if x >= 1 else 0)

# Convert Values to Z Scores
z_city_cols = [
       "Population",
       "Elevation: Median",
       "Weather: Mean Max Temp"
]

z_vals = cities_df[z_city_cols].apply(zscore)

cities_df = cities_df.join(z_vals, rsuffix=" (Z)")

# Reverse Values where Appropriate - When Low Values indicate Higher Attractiveness
cities_df["LC: Ag % (Inv)"] = 1 - cities_df["LC: Ag %"]
cities_df["Elevation: Median (Z - Inv)"] = -1 * cities_df["Elevation: Median (Z)"]

# Make Z Values Positive
cities_df["Population (Z)"] = cities_df["Population (Z)"].apply(lambda x: abs(cities_df["Population (Z)"].min()) + x)
cities_df["Elevation: Median (Z - Inv)"] = cities_df["Elevation: Median (Z - Inv)"].apply(lambda x: abs(cities_df["Elevation: Median (Z - Inv)"].min()) + x)
cities_df["Weather: Mean Max Temp (Z)"] = cities_df["Weather: Mean Max Temp (Z)"].apply(lambda x: abs(cities_df["Weather: Mean Max Temp (Z)"].min()) + x)

# Drop Columns
drop_city_cols = [
       "Shape Length",
       "County",
       "GNIS",
       "Weather: Join Count",
       "Weather: Mean Min Temp",
       "LC: Urban",
       "LC: Ag",
       "LC: Natural",
       "Observations: Count",
       "SHAPELENGTH",
       "SHAPEAREA",
       "Elevation: Mean",
       "Elevation: Range",
       "Elevation: StDev",
       "Weather: Mean Precip",
       "LC: Natural %",
       "Population",
       "Elevation: Median",
       "Weather: Mean Max Temp",
       "LC: Ag %",
       "Elevation: Median (Z)"
]

cities_df = cities_df.drop(drop_city_cols, axis=1)

In [3]:
# Calculate Weights/Probabilities
cities_df["Wi"] = cities_df[["LC: Urban %", "LC: Ag % (Inv)", "Population (Z)", "Elevation: Median (Z - Inv)", "Weather: Mean Max Temp (Z)"]].sum(axis=1)

cities_df["Wi"].describe()

count    903.000000
mean       8.380871
std        2.181309
min        2.600151
25%        6.864827
50%        8.310738
75%        9.573658
max       30.939597
Name: Wi, dtype: float64

In [4]:
# Drop Unneeded Columns
cities_df = cities_df[["OBJECTID", "City", "Wi", "Observations: Presence"]].copy()

cities_df

,OBJECTID,City,Wi,Observations: Presence
0,1,Bluffton,5.969539,0
1,2,Sartell,9.284543,0
2,3,Cambridge,9.502162,1
3,4,Waseca,9.172596,0
4,5,La Crescent,11.639762,0
...,...,...,...,...
898,899,Plainview,8.524764,0
899,900,Albertville,11.816777,0
900,901,Storden,6.753598,0
901,902,Rosemount,10.907398,1


### Load Lags

In [5]:
lags_df = pd.read_csv("../data/model/distances.csv")

lags_df = lags_df.drop(["OID_", "Shape_Length"], axis=1)

lags_df

,ORIG_FID,DEST_FID,LINK_DIST
0,1,34,43.862782
1,1,38,41.582255
2,1,42,13.030666
3,1,47,56.200086
4,1,52,47.859184
...,...,...,...
90295,903,858,19.696690
90296,903,868,4.809337
90297,903,871,7.856454
90298,903,880,23.162931


### Join Lags & Attributes

In [6]:
# Join 
final_df = lags_df.merge(cities_df, left_on="ORIG_FID", right_on="OBJECTID")
final_df = final_df.merge(cities_df, left_on="DEST_FID", right_on="OBJECTID", suffixes=("", "_TO"))

# Drop Columns
final_df = final_df.drop(["ORIG_FID", "DEST_FID", "OBJECTID", "OBJECTID_TO", "Observations: Presence_TO"], axis=1)

# Add Empty End Presence Field
final_df["BMSB Presence: j"] = 0

# Rename Columns
final_df.columns = ["Distance", "City: From", "W: From", "BMSB Presence: From", "City: To", "W: To", "BMSB Presence: To"]

# Drop Rows Where To/From are Same
final_df = final_df[final_df["City: From"] != final_df["City: To"]]

# Drop Long Distances
final_df = final_df.loc[final_df["Distance"] < 50]

final_df

,Distance,City: From,W: From,BMSB Presence: From,City: To,W: To,BMSB Presence: To
0,43.862782,Bluffton,5.969539,0,Randall,7.135679,0
1,35.085990,Sartell,9.284543,0,Randall,7.135679,0
2,48.574812,Aitkin,7.276592,0,Randall,7.135679,0
4,37.975182,Crosby,7.112552,0,Randall,7.135679,0
5,42.119042,Elrosa,7.085966,0,Randall,7.135679,0
...,...,...,...,...,...,...,...
90293,40.077688,Saint Leo,9.081634,0,Odessa,9.711266,0
90295,9.014127,Bellingham,9.554981,0,Odessa,9.711266,0
90297,21.773488,Graceville,8.976602,0,Odessa,9.711266,0
90298,47.188037,Kensington,7.513153,0,Odessa,9.711266,0


## Run Simulation

In [13]:
# Run Simple Huff Model - No Distance Decay
huff_simple = Simulation(final_df)

# Run 100 Iterations
hs_df = huff_simple.monte_carlo("HUFF_SIMPLE", 100, True)

hs_df

100%|██████████| 100/100 [02:32<00:00,  1.53s/it]


,Distance,City: From,W: From,BMSB Presence: From,City: To,W: To,BMSB Presence: To,HS: Wi/Dij,Huff: Simple,HS: Transition Count
0,43.862782,Bluffton,5.969539,1,Randall,7.135679,0,0.162682,0.000494,0
1,35.085990,Sartell,9.284543,1,Randall,7.135679,0,0.203377,0.000618,0
2,48.574812,Aitkin,7.276592,1,Randall,7.135679,0,0.146901,0.000446,0
4,37.975182,Crosby,7.112552,0,Randall,7.135679,0,0.187904,0.000571,0
5,42.119042,Elrosa,7.085966,1,Randall,7.135679,0,0.169417,0.000515,0
...,...,...,...,...,...,...,...,...,...,...
90293,40.077688,Saint Leo,9.081634,1,Odessa,9.711266,0,0.242311,0.000736,0
90295,9.014127,Bellingham,9.554981,0,Odessa,9.711266,0,1.077339,0.003273,0
90297,21.773488,Graceville,8.976602,0,Odessa,9.711266,0,0.446013,0.001355,0
90298,47.188037,Kensington,7.513153,0,Odessa,9.711266,0,0.205799,0.000625,0


In [7]:
# Run Huff Model - With Distance Decay (alpha = 2)
huff_decay = Simulation(final_df)

# Run 100 Iterations
hd2_df = huff_decay.monte_carlo("HUFF_DECAY", 100, True)

hd2_df

100%|██████████| 100/100 [05:27<00:00,  3.27s/it]


,Distance,City: From,W: From,BMSB Presence: From,City: To,W: To,BMSB Presence: To,HD2: Wi/Dij,Huff: Decay of 2,HD2: Transition Count
0,43.862782,Bluffton,5.969539,0,Randall,7.135679,0,0.003709,0.000094,0
1,35.085990,Sartell,9.284543,0,Randall,7.135679,0,0.005797,0.000147,0
2,48.574812,Aitkin,7.276592,0,Randall,7.135679,0,0.003024,0.000076,0
4,37.975182,Crosby,7.112552,0,Randall,7.135679,0,0.004948,0.000125,0
5,42.119042,Elrosa,7.085966,0,Randall,7.135679,0,0.004022,0.000102,0
...,...,...,...,...,...,...,...,...,...,...
90293,40.077688,Saint Leo,9.081634,0,Odessa,9.711266,0,0.006046,0.000153,0
90295,9.014127,Bellingham,9.554981,0,Odessa,9.711266,0,0.119517,0.003023,0
90297,21.773488,Graceville,8.976602,0,Odessa,9.711266,0,0.020484,0.000518,0
90298,47.188037,Kensington,7.513153,0,Odessa,9.711266,0,0.004361,0.000110,0


In [15]:
# Run Gravity Model - Converted to Probability
gravity = Simulation(final_df)

# Run 100 Iterations
g_df = gravity.monte_carlo("GRAVITY_SIMPLE", 100, True)

g_df

100%|██████████| 100/100 [02:32<00:00,  1.52s/it]


,Distance,City: From,W: From,BMSB Presence: From,City: To,W: To,BMSB Presence: To,Gravity,Gravity: Probability,G: Transition Count
0,43.862782,Bluffton,5.969539,0,Randall,7.135679,0,0.971136,0.000310,0
1,35.085990,Sartell,9.284543,1,Randall,7.135679,0,1.888262,0.000603,0
2,48.574812,Aitkin,7.276592,0,Randall,7.135679,0,1.068937,0.000341,0
4,37.975182,Crosby,7.112552,0,Randall,7.135679,0,1.336475,0.000427,0
5,42.119042,Elrosa,7.085966,1,Randall,7.135679,0,1.200483,0.000383,0
...,...,...,...,...,...,...,...,...,...,...
90293,40.077688,Saint Leo,9.081634,1,Odessa,9.711266,0,2.200580,0.000703,0
90295,9.014127,Bellingham,9.554981,1,Odessa,9.711266,0,10.293949,0.003286,0
90297,21.773488,Graceville,8.976602,0,Odessa,9.711266,0,4.003684,0.001278,0
90298,47.188037,Kensington,7.513153,0,Odessa,9.711266,0,1.546202,0.000494,0


## Aggregate Results to Cities

In [8]:
# Total Incoming Bugs per City - one way we can measure hazard
hs_df_incoming = hs_df.groupby("City: To").agg(["sum"])["HS: Transition Count"]
hs_df_incoming.columns = ["HSIn"]

hd2_df_incoming = hd2_df.groupby("City: To").agg(["sum"])["HD2: Transition Count"]
hd2_df_incoming.columns = ["HD2In"]

g_df_incoming = g_df.groupby("City: To").agg(["sum"])["G: Transition Count"]
g_df_incoming.columns = ["GIn"]

In [9]:
# Total Outgoing Bugs per City - another way we can measure hazard
hs_df_outgoing = hs_df.groupby("City: From").agg(["sum"])["HS: Transition Count"]
hs_df_outgoing.columns = ["HSOut"]

hd2_df_outgoing = hd2_df.groupby("City: From").agg(["sum"])["HD2: Transition Count"]
hd2_df_outgoing.columns = ["HD2Out"]

g_df_outgoing = g_df.groupby("City: From").agg(["sum"])["G: Transition Count"]
g_df_outgoing.columns = ["GOut"]

In [10]:
# Total Percentage Risk to City - another way we can measure hazard
hs_df["Huff: Simple"] /= 100
hs_df_sum_prob = hs_df.groupby("City: To").agg(["sum"])["Huff: Simple"]
hs_df_sum_prob.columns = ["HSRisk"]

hd2_df["Huff: Decay of 2"] /= 100
hd2_df_sum_prob = hd2_df.groupby("City: To").agg(["sum"])["Huff: Decay of 2"]
hd2_df_sum_prob.columns = ["HD2Risk"]

g_df["Gravity: Probability"] /= 100
g_df_sum_prob = g_df.groupby("City: To").agg(["sum"])["Gravity: Probability"]
g_df_sum_prob.columns = ["GRisk"]

In [11]:
# Merge Aggregated DFs Together
hs_final = hs_df_incoming.merge(hs_df_outgoing, left_index=True, right_index=True).merge(hs_df_sum_prob, left_index=True, right_index=True)

hd2_final = hd2_df_incoming.merge(hd2_df_outgoing, left_index=True, right_index=True).merge(hd2_df_sum_prob, left_index=True, right_index=True)

g_final = g_df_incoming.merge(g_df_outgoing, left_index=True, right_index=True).merge(g_df_sum_prob, left_index=True, right_index=True)

In [12]:
# Read in City Shapefile
city_shp = gpd.read_file("../data/cities/city_township_unorg.shp")

# Get Cities Only
city_shp = city_shp.loc[city_shp["CTU_CLASS"] == "CITY"]

# Convert to Centroids
city_shp["centroid"] = city_shp.centroid
city_shp.set_geometry("centroid")

# Drop Unneccesary Columns
city_gdf = city_shp[["FEATURE_NA", "centroid"]].copy()

city_gdf.head()

,FEATURE_NA,centroid
2,Bluffton,POINT (328504.432 5148648.358)
6,Sartell,POINT (404522.736 5052398.100)
7,Cambridge,POINT (482213.265 5045345.354)
8,Waseca,POINT (459657.376 4881117.971)
9,La Crescent,POINT (636366.575 4854264.051)


In [13]:
# Join Results to GDF
hs_gdf = city_gdf.merge(hs_final, left_on="FEATURE_NA", right_index=True)
hd2_gdf = city_gdf.merge(hd2_final, left_on="FEATURE_NA", right_index=True)
g_gdf = city_gdf.merge(g_final, left_on="FEATURE_NA", right_index=True)

# Rename Columns
hs_gdf.columns = ["City", "geom", "Incoming", "Outgoing", "Risk"]
hd2_gdf.columns = ["City", "geom", "Incoming", "Outgoing", "Risk"]
g_gdf.columns = ["City", "geom", "Incoming", "Outgoing", "Risk"]

In [17]:
# Ensure that Duplicates are Dropped
hs_gdf = hs_gdf.drop_duplicates()
hd2_gdf = hd2_gdf.drop_duplicates()
g_gdf = g_gdf.drop_duplicates()

## Upload Results to PostgreSQL Database

In [18]:
# Create Engine to Connect to PostgreSQL Database
engine = create_engine("<CONNECTION_STRING_HERE>", echo=False)

In [75]:
# Convert Point Objects to WKT Format
hs_gdf['geom'] = hs_gdf['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))

# Write GeoDataFrame to Database
hs_gdf.to_sql("huff_model", engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid=4326)})

902

In [19]:
# Convert Point Objects to WKT Format
hd2_gdf['geom'] = hd2_gdf['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))

# Write GeoDataFrame to Database
hd2_gdf.to_sql("huff_model_distance_decay", engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid=4326)})

902

In [81]:
# Convert Point Objects to WKT Format
g_gdf['geom'] = g_gdf['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))

# Write GeoDataFrame to Database
g_gdf.to_sql("gravity_model", engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid=4326)})

902